In [2]:
from src.utils import create_inception_dataset
from src.utils import prepare_kanji_dict
# prepare_kanji_dict(regenerate=True, resolution=512)

create_inception_dataset(push_to_hub=True, hub_model_id="Ksgk-fy/inception-kanji-dataset-512")


Batch 7: 100%|██████████| 434/434 [00:03<00:00, 127.29it/s]


Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/15625 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/157 [00:00<?, ?ba/s]

Map:   0%|          | 0/15625 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/157 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'image'],
        num_rows: 31250
    })
})

In [1]:
!python prepare_data.py --push_to_hub --max_n 2 --hub_model_id Ksgk-fy/inception-kanji-dataset-512

Traceback (most recent call last):
  File "/Users/fangyuanyu/Grow/DiffuseDraw/prepare_data.py", line 1, in <module>
    from src.utils import create_inception_dataset
  File "/Users/fangyuanyu/Grow/DiffuseDraw/src/__init__.py", line 1, in <module>
    from .utils import *
  File "/Users/fangyuanyu/Grow/DiffuseDraw/src/utils.py", line 4, in <module>
    from cairosvg import svg2png
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/cairosvg/__init__.py", line 26, in <module>
    from . import surface  # noqa isort:skip
    ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/cairosvg/surface.py", line 9, in <module>
    import cairocffi as cairo
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/cairocffi/__init__.py", line 60, in <module>
    cairo = dlopen(
            ^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/cairocffi/__init__.py", line 57, in dlopen
    raise OSError(error_message)  # pragma: no cover
    

In [4]:
svg_path = "data/kanji/0003f.svg"
png_path = "test.png"

from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM

def convert_svg_to_png(svg_filepath, png_filepath):
    drawing = svg2rlg(svg_filepath)
    renderPM.drawToFile(drawing, png_filepath, fmt="PNG")

# Example usage:
convert_svg_to_png(svg_path, png_path)


# import os 
# command_str = f"rsvg-convert --format=png --output={png_path} {svg_path}"
# os.system(command_str)


RenderPMError: cannot import desired renderPM backend rlPyCairo
Seek advice at the users list see
https://pairlist2.pair.net/mailman/listinfo/reportlab-users

In [ ]:
# caption meaning is not clean English -- need to filter it 


In [1]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("Ksgk-fy/kanji_inception_finetune_v4")
pipe.to("cuda")
prompt = "a Kanji meaning Elon Musk"
image = pipe(prompt).images[0]
    

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


model_index.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

Fetching 35 files:   0%|          | 0/35 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
images[7].save("donaldtrump.png")
image = add_caption(images[7], "Donald Trump")
image.save("Donald_Trump.png")

import matplotlib.pyplot as plt

def plot_images(images, captions=None, rows=None, cols=None):
    n = len(images)
    if rows is None and cols is None:
        cols = min(4, n)
        rows = (n + cols - 1) // cols
    elif rows is None:
        rows = (n + cols - 1) // cols
    elif cols is None:
        cols = (n + rows - 1) // rows
        
    fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
    if rows == 1 and cols == 1:
        axes = [[axes]]
    elif rows == 1 or cols == 1:
        axes = [axes]
        
    for i in range(rows):
        for j in range(cols):
            idx = i * cols + j
            if idx < n:
                axes[i][j].imshow(images[idx])
                if captions:
                    axes[i][j].set_title(captions[idx])
            axes[i][j].axis('off')
            
    plt.tight_layout()
    plt.show()

# Example usage:
# plot_images(images) # To plot all images
# plot_images(images, rows=2, cols=4) # To specify layout